# Visualizing source activities

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import numpy as np

In [ ]:
# SETUP PATHS
src_dir = Path("/work/neuro/sources")
sub_files = list(src_dir.glob('*'))
print(f"sub_files: {sub_files}")

statistics = {}

for i, file in enumerate(sub_files):
    print(f"Processing file {i}, path {file}")
    base = joblib.load(file)
    # inspect contents
    subject_id = base['subject_id']
    
    xs=base['Xs']
    lobes = ['occipital', 'frontal']
    y = base['y']

    for lobe in lobes:
        scts = xs[lobe]
        lobe_per_epoch = np.mean(scts, axis=1)  # axis=1 is sources
        classes = np.unique(y)

        results = {}

        for c in classes:
            # Boolean mask for epochs in class c
            mask = (y == c)
            
            # Extract epochs for this class: (n_epochs_c, 1201)
            epochs_c = lobe_per_epoch[mask]  # shape: (n_c, 1201)
            
            # Skip if no epochs
            if epochs_c.shape[0] == 0:
                print(f"skip no epoch")
                continue
                
            # Mean and SEM across epochs (within class)
            mean_c = epochs_c.mean(axis=0)  # (1201,)
            sem_c  = epochs_c.std(axis=0, ddof=1) / np.sqrt(epochs_c.shape[0])
            
            results[c] = {
                'mean': mean_c,
                'sem': sem_c,
                'n_epochs': epochs_c.shape[0]
            }

        if lobe not in statistics:
            statistics[lobe] = {}  # create dict for this lobe
        statistics[lobe][subject_id] = results
    

Creating a side-by-side subplot

In [ ]:
# Extract time vector (assuming same for all subjects)
times = np.linspace(-0.2, 1.0, 1201)  # adjust to your actual time

# Create figure with subplots for both lobes side by side
fig, axes = plt.subplots(1, 2, figsize=(15, 5))  # 15-inch width for standard page

# Loop over each lobe
for idx, lobe in enumerate(statistics.keys()):
    print(f"\nPlotting {lobe} lobe...")
    
    # Choose colormap based on lobe with better color ranges
    if lobe == 'occipital':
        color_map = plt.cm.Blues  # Dark blue to light blue
    elif lobe == 'frontal':
        color_map = plt.cm.Reds   # Dark red to light red/orange
    else:
        color_map = plt.cm.tab10  # Default for other lobes
    
    # Collect all subjects' data for this lobe
    class_data = {}
    
    for subject_id, subject_data in statistics[lobe].items():
        for class_id, class_stats in subject_data.items():
            if class_id not in class_data:
                class_data[class_id] = []
            class_data[class_id].append(class_stats['mean'])  # shape: (1201,)
    
    # Convert to arrays and compute group mean ± SEM
    group_results = {}
    for class_id, means_list in class_data.items():
        # Stack subject means: (n_subjects, 1201)
        subj_means = np.stack(means_list, axis=0)
        
        # Group mean and SEM across subjects
        group_mean = subj_means.mean(axis=0)  # (1201,)
        group_sem  = subj_means.std(axis=0, ddof=1) / np.sqrt(subj_means.shape[0])
        
        group_results[class_id] = {
            'mean': group_mean,
            'sem': group_sem,
            'n_subjects': subj_means.shape[0]
        }
    
    # Create color gradient with more contrast for this lobe's classes
    n_classes = len(group_results)
    # Generate distinct colors for each class within the lobe's color map
    if n_classes == 1:
        class_colors = [color_map(0.8)]
    elif n_classes == 2:
        class_colors = [color_map(0.25), color_map(0.8)]  # Dark to light
    elif n_classes == 3:
        class_colors = [color_map(0.2), color_map(0.5), color_map(0.8)]
    elif n_classes == 4:
        class_colors = [color_map(0.4), color_map(0.6), color_map(0.8), color_map(.99)]
    else:
        class_colors = color_map(np.linspace(0.15, 0.9, n_classes))
    
    labels=["NE - 1", "WG - 2", "ACE - 3", "CE - 4"]
    for i, c in enumerate(sorted(group_results)):
        mean_c = group_results[c]['mean']
        sem_c  = group_results[c]['sem']
        
        axes[idx].plot(times, mean_c, label=labels[i], color=class_colors[i])
        axes[idx].fill_between(times, mean_c - sem_c, mean_c + sem_c, 
                               color=class_colors[i], alpha=0.1)

    axes[idx].axvline(0, color='k', linestyle='--', linewidth=0.8)
    axes[idx].set_xlabel('Time (s)')
    axes[idx].set_ylabel(f'Source amplitude (A·m)')
    
    # Set x-axis ticks every 100 ms
    tick_interval = 0.1  # seconds
    xticks = np.arange(times[0], times[-1]+tick_interval, tick_interval)
    axes[idx].set_xticks(xticks)

    # Make plot minimal - remove top and right spines
    axes[idx].spines['top'].set_visible(False)
    axes[idx].spines['right'].set_visible(False)
    
    axes[idx].legend(loc='upper left')
    axes[idx].set_title(f'{lobe.capitalize()} lobe: Group-level evoked response')

plt.tight_layout()
plt.savefig("plots/both_lobes_average.png", dpi=300, bbox_inches='tight')
plt.show()

# Print subject counts per class for each lobe
for lobe in statistics.keys():
    print(f"\nSubjects per class in {lobe}:")
    class_data = {}
    for subject_id, subject_data in statistics[lobe].items():
        for class_id, class_stats in subject_data.items():
            if class_id not in class_data:
                class_data[class_id] = []
            class_data[class_id].append(class_stats['mean'])
    
    for class_id, means_list in class_data.items():
        print(f"  Class {class_id}: n={len(means_list)}")